In [ ]:
import akshare as ak

In [ ]:
ak.__version__

# stock

## 获取股票详情

### polars

In [ ]:
import polars as pl

In [ ]:
import typing
import arrow

def get_by_stock(code: str, date: typing.Optional[str]= None):
    if not date:
        date = arrow.now().format("YYYYMMDD")
    stock_zh_a_hist_df = ak.stock_zh_a_hist(
        symbol=code,
        period="daily",
        start_date=date,
        end_date=date,
        adjust="",
    )

    stock = pl.DataFrame(stock_zh_a_hist_df)
    return stock.select(
        pl.col("日期").dt.strftime("%Y%m%d").alias("date"),
        pl.col("股票代码").alias("code"),
        pl.col("开盘").alias("open"),
        pl.col("收盘").alias("close"),
        pl.col("最高").alias("high"),
        pl.col("最低").alias("low"),
        pl.col("成交量").alias("volumn"),
        pl.col("成交额").alias("transaction_volume"),
        pl.col("涨跌幅").alias("change_pct"),
        pl.col("换手率").alias("turnover_rate"),
    )[0].to_dicts()[0]

In [ ]:
get_by_stock("300251")

### pandas

In [ ]:
import pandas as pd

def get_stock_data(code, date=None):
    if not date:
        date = arrow.now().format("YYYYMMDD")

    # 获取 A 股历史数据
    stock_zh_a_hist_df = ak.stock_zh_a_hist(
        symbol=code,
        period="daily",
        start_date=date,
        end_date=date,
        adjust="",
    )

    # 选取并重命名列
    stock = stock_zh_a_hist_df.rename(
        columns={
            "日期": "date",
            "股票代码": "code",
            "开盘": "open",
            "收盘": "close",
            "最高": "high",
            "最低": "low",
            "成交量": "volumn",
            "成交额": "transaction_volume",
            "涨跌幅": "change_pct",
            "换手率": "turnover_rate",
            "振幅": "amplitude",
            "振幅": "amplitude",
        }
    )

    # 转换日期格式
    stock["date"] = pd.to_datetime(stock["date"]).dt.strftime("%Y%m%d")
    stock = stock.drop(columns=["涨跌额"])
    # 获取第一行数据并转换为字典
    return stock.iloc[0].to_dict()

In [ ]:
get_stock_data("300251")

## 获取全部股票

In [ ]:
import akshare as ak

data=ak.stock_zh_a_spot_em()
filtered_df = data[data["代码"].str.contains("光线", na=False)]
df_selected = filtered_df[["代码", "名称", "最新价"]].rename(
    columns={"代码": "code", "名称": "name", "最新价": "price"}
)
df_selected


# news

In [ ]:
news=ak.stock_news_em(symbol="000002")

In [ ]:
# 获取最新的一条
newest = news.sort_values(by="发布时间", ascending=False).iloc[0]
newest

In [ ]:
newest["新闻内容"]